# **Testing the following process: Loading, Merging, Wrangling and Consolidating Data**

- Load the datasets (SPY, features from yf and FRED)
- Drop the first 2 rows of features loaded from yf
- Rename column 'Date' and convert it into datetime
- Set 'Date' as index
- Apply the numeric transformation for datasets
- Drop the unnecessary columns for FRED datasets 
- Keep only the 'Close' price columns for assets (not SPY data) 
- Merge the features dataframe (data_fin and data_FRED)

In [120]:
# Necessary libraries

import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path

In [121]:
# Adjust directories (sources and outputs)

load_dotenv()

raw_data_path = os.getenv("RAW_DATA_PATH")
processed_data_path = os.getenv("PROCESSED_DATA_PATH")

raw_data_path = Path(raw_data_path)
processed_data_path = Path(processed_data_path)

# SPY Data

In [122]:
# Main feature - S&P 500 ETF (SPY)

spy = pd.read_csv(raw_data_path / 'SPY_raw_data.csv', header = 0)

spy = spy.iloc[2:].reset_index(drop = True)
spy = spy.rename(columns = {spy.columns[0]: 'Date'})
spy['Date'] = pd.to_datetime(spy['Date'])
spy = spy.set_index('Date')
spy = spy.apply(pd.to_numeric, errors = 'coerce')

print(spy.info())
print("--" * 30)

spy.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5256 entries, 2005-01-03 to 2025-11-20
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   5256 non-null   float64
 1   High    5256 non-null   float64
 2   Low     5256 non-null   float64
 3   Open    5256 non-null   float64
 4   Volume  5256 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 246.4 KB
None
------------------------------------------------------------


,Close,High,Low,Open,Volume
Date,,,,,
2005-01-03,81.847115,82.840437,81.574970,82.704362,55748000
2005-01-04,80.847000,82.010413,80.581661,81.955983,69167600
2005-01-05,80.289101,81.132744,80.282296,80.785759,65667300
2005-01-06,80.697327,81.064721,80.459202,80.581667,47814700
2005-01-07,80.581680,81.119164,80.370766,80.942270,55847700


# Financial features (other assets) Function

In [123]:
# Function to load and clean CSV files 

def load_and_clean_tickers(path):

    df = pd.read_csv(path, header = 0)

    df = df.iloc[2:].reset_index(drop=True)
    df = df.rename(columns={df.columns[0]: 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df = df.apply(pd.to_numeric, errors = 'coerce')

    df = df['Close'].to_frame()

    return df

In [124]:
# Invoke the fuction

tickers = {
    "vix": "VIX_raw_data.csv",
    "gold": "Gold_raw_data.csv",
    "oil": "CrudeOil_raw_data.csv",
    "tlt": "TLT_raw_data.csv",
    "rsp": "RSP_raw_data.csv",
    "tnx": "TNX_raw_data.csv",
    "iwm": "IWM_raw_data.csv",
    "dxy": "DXY_raw_data.csv"
}

data_fin = {name: load_and_clean_tickers(raw_data_path / file) for name, file in tickers.items()}

In [125]:
# Convert the dictionary into a dataframe

data_fin = pd.concat(
    {k: v.rename(columns = {'Close': k}) for k, v in data_fin.items()},
    axis = 1
)

data_fin.columns = data_fin.columns.droplevel(0)
data_fin


,vix,gold,oil,tlt,rsp,tnx,iwm,dxy
Date,,,,,,,,
2005-01-03,14.080000,428.700012,42.119999,45.395805,27.763653,4.220,48.436901,81.300003
2005-01-04,13.980000,428.500000,43.910000,44.920033,27.376051,4.283,47.398598,82.570000
2005-01-05,14.090000,426.600006,43.389999,45.160500,27.165127,4.277,46.451237,82.540001
2005-01-06,13.580000,421.000000,45.560001,45.191257,27.246248,4.272,46.697548,83.150002
2005-01-07,13.490000,418.899994,45.430000,45.293507,27.174141,4.285,46.178391,83.610001
...,...,...,...,...,...,...,...,...
2025-11-14,19.830000,4087.600098,60.090000,88.870003,187.339996,4.148,237.479996,99.269997
2025-11-17,22.379999,4068.300049,59.910000,89.089996,184.880005,4.133,232.759995,99.589996
2025-11-18,24.690001,4061.300049,60.740002,89.059998,184.990005,4.123,233.470001,99.550003


In [126]:
print(data_fin.isnull().sum())
print("--" * 30)
data_fin.info()

vix     10
gold    15
oil     11
tlt     10
rsp     10
tnx     15
iwm     10
dxy      4
dtype: int64
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5266 entries, 2005-01-03 to 2025-11-20
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   vix     5256 non-null   float64
 1   gold    5251 non-null   float64
 2   oil     5255 non-null   float64
 3   tlt     5256 non-null   float64
 4   rsp     5256 non-null   float64
 5   tnx     5251 non-null   float64
 6   iwm     5256 non-null   float64
 7   dxy     5262 non-null   float64
dtypes: float64(8)
memory usage: 370.3 KB


# FRED Data Function

In [127]:
# Function to load and clean CSV files from FRED sources

def load_csv_FRED(path, column):

    df = pd.read_csv(path, parse_dates=['Date'], index_col = 'Date')

    return df[[column]]

In [128]:
# Invoke the function 

datasets_FRED = {
    "baa10yc": ("Baa_Corporate_to_10_Yield.csv", "BAA10Y"),
    "corp710y": ("Corporate_Bond_710_raw_data.csv", "BAMLC4A0C710YEY"),
    "nfci": ("NFCI_fin_condition_raw_data.csv", "NFCI"),
    "str_index": ("STLFSI4_Stress_raw_data.csv", "STLFSI4"),
    "t5yie": ("T5YIE_Breakeven_raw_data.csv", "T5YIE"),
    "t10y2y": ("T10Y_minus_2Y_raw_data.csv", "T10Y2Y"),
    "t10y3m": ("T10Y_minus_3M_raw_data.csv", "T10Y3M"),
    "effr": ("EFFR_funds_rates_raw_data.csv", "EFFR"),
    "high_yield": ("High_Yield_raw_data.csv", "BAMLH0A0HYM2")
}

data_FRED = {name: load_csv_FRED(raw_data_path / file[0], file[1]) for name, file in datasets_FRED.items()}

In [129]:
# Convert the dictionary into a dataframe

data_FRED = pd.concat(
    {k: v.rename(columns = {'Close': k}) for k, v in data_FRED.items()},
    axis = 1
)

data_FRED.columns = data_FRED.columns.droplevel(0)
data_FRED

,BAA10Y,BAMLC4A0C710YEY,NFCI,STLFSI4,T5YIE,T10Y2Y,T10Y3M,EFFR,BAMLH0A0HYM2
Date,,,,,,,,,
2005-01-03,1.86,4.94,NaN,NaN,2.62,1.13,1.91,2.31,3.06
2005-01-04,1.85,5.01,NaN,NaN,2.62,1.09,1.96,2.25,2.97
2005-01-05,1.83,5.00,NaN,NaN,2.60,1.07,1.96,2.25,3.04
2005-01-06,1.84,5.00,NaN,NaN,2.59,1.11,1.98,2.25,3.09
2005-01-07,1.83,5.02,-0.69875,-0.7361,2.57,1.09,1.97,2.24,3.07
...,...,...,...,...,...,...,...,...,...
2025-11-17,1.77,4.99,NaN,NaN,2.37,0.53,0.16,3.88,3.13
2025-11-18,1.79,4.98,NaN,NaN,2.36,0.54,0.18,3.88,3.20
2025-11-19,1.79,4.99,NaN,NaN,2.36,0.55,0.18,3.88,3.17


In [130]:
print(data_FRED.isnull().sum())
print("--" * 30)
data_FRED.info()

BAA10Y              296
BAMLC4A0C710YEY      64
NFCI               4429
STLFSI4            4429
T5YIE               292
T10Y2Y              292
T10Y3M              292
EFFR                272
BAMLH0A0HYM2         64
dtype: int64
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5519 entries, 2005-01-03 to 2025-11-21
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BAA10Y           5223 non-null   float64
 1   BAMLC4A0C710YEY  5455 non-null   float64
 2   NFCI             1090 non-null   float64
 3   STLFSI4          1090 non-null   float64
 4   T5YIE            5227 non-null   float64
 5   T10Y2Y           5227 non-null   float64
 6   T10Y3M           5227 non-null   float64
 7   EFFR             5247 non-null   float64
 8   BAMLH0A0HYM2     5455 non-null   float64
dtypes: float64(9)
memory usage: 431.2 KB


The "NFCI (National Financial Condition Index)" and "STLFSI4 (Stress Level Index)" columns contain a large number of null (NaN) values. This occurs because both series are originally reported at a weekly frequency, so when they are incorporated into a dataset with daily frequency, gaps naturally appear on the days between observations.

To properly handle these missing values, the following procedure will be applied:

  * Apply forward filling (ffill()) to propagate the most recent available value forward until a new weekly observation is encountered.

This approach preserves the temporal structure of the original weekly data while adapting the series to the daily frequency required for the analysis, without introducing artificial or inconsistent values. With that, the data will talk the same language, daily frequency, but not exactly because the market stays close in weekends, holidays and other celebrations.

In [131]:
# Apply the previous strategy to manage the columns with different frequencies (using forward fill)

def apply_ffill(df, weekly_columns):

    # Make sure the index is in datetime format
    
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    # Aplly the forward fill

    df[weekly_columns] = df[weekly_columns].ffill()

    return df

In [132]:
# Invoke the function

# For data_fin

# There are no weekly columns in data_fin

# For data_FRED

weekly_cols_FRED = ['NFCI', 'STLFSI4']   

data_FRED = apply_ffill(df = data_FRED, weekly_columns = weekly_cols_FRED)
data_FRED

,BAA10Y,BAMLC4A0C710YEY,NFCI,STLFSI4,T5YIE,T10Y2Y,T10Y3M,EFFR,BAMLH0A0HYM2
Date,,,,,,,,,
2005-01-03,1.86,4.94,NaN,NaN,2.62,1.13,1.91,2.31,3.06
2005-01-04,1.85,5.01,NaN,NaN,2.62,1.09,1.96,2.25,2.97
2005-01-05,1.83,5.00,NaN,NaN,2.60,1.07,1.96,2.25,3.04
2005-01-06,1.84,5.00,NaN,NaN,2.59,1.11,1.98,2.25,3.09
2005-01-07,1.83,5.02,-0.69875,-0.7361,2.57,1.09,1.97,2.24,3.07
...,...,...,...,...,...,...,...,...,...
2025-11-17,1.77,4.99,-0.53304,-0.5071,2.37,0.53,0.16,3.88,3.13
2025-11-18,1.79,4.98,-0.53304,-0.5071,2.36,0.54,0.18,3.88,3.20
2025-11-19,1.79,4.99,-0.53304,-0.5071,2.36,0.55,0.18,3.88,3.17


# Merging the data and Consolidating the Features


The next step is to consolidate the features dataframe, which will contain data_fin and data_FRED dataframes, without cleaning and processign the final version of features.

In [133]:
print(len(data_fin))
print(len(data_FRED))

5266
5519


In [134]:
# Merging all the data (data_fin and data_FRED) and consolidating into a single dataframe (data_features)

data_features = data_fin.join(data_FRED, how = 'left')

print(data_features.isnull().sum())
print("--" * 30)
data_features

vix                10
gold               15
oil                11
tlt                10
rsp                10
tnx                15
iwm                10
dxy                 4
BAA10Y             49
BAMLC4A0C710YEY     2
NFCI                4
STLFSI4             4
T5YIE              47
T10Y2Y             47
T10Y3M             47
EFFR               49
BAMLH0A0HYM2        2
dtype: int64
------------------------------------------------------------


,vix,gold,oil,tlt,rsp,tnx,iwm,dxy,BAA10Y,BAMLC4A0C710YEY,NFCI,STLFSI4,T5YIE,T10Y2Y,T10Y3M,EFFR,BAMLH0A0HYM2
Date,,,,,,,,,,,,,,,,,
2005-01-03,14.080000,428.700012,42.119999,45.395805,27.763653,4.220,48.436901,81.300003,1.86,4.94,NaN,NaN,2.62,1.13,1.91,2.31,3.06
2005-01-04,13.980000,428.500000,43.910000,44.920033,27.376051,4.283,47.398598,82.570000,1.85,5.01,NaN,NaN,2.62,1.09,1.96,2.25,2.97
2005-01-05,14.090000,426.600006,43.389999,45.160500,27.165127,4.277,46.451237,82.540001,1.83,5.00,NaN,NaN,2.60,1.07,1.96,2.25,3.04
2005-01-06,13.580000,421.000000,45.560001,45.191257,27.246248,4.272,46.697548,83.150002,1.84,5.00,NaN,NaN,2.59,1.11,1.98,2.25,3.09
2005-01-07,13.490000,418.899994,45.430000,45.293507,27.174141,4.285,46.178391,83.610001,1.83,5.02,-0.69875,-0.7361,2.57,1.09,1.97,2.24,3.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-14,19.830000,4087.600098,60.090000,88.870003,187.339996,4.148,237.479996,99.269997,1.77,4.99,-0.53304,-0.5071,2.38,0.52,0.19,3.88,3.07
2025-11-17,22.379999,4068.300049,59.910000,89.089996,184.880005,4.133,232.759995,99.589996,1.77,4.99,-0.53304,-0.5071,2.37,0.53,0.16,3.88,3.13
2025-11-18,24.690001,4061.300049,60.740002,89.059998,184.990005,4.123,233.470001,99.550003,1.79,4.98,-0.53304,-0.5071,2.36,0.54,0.18,3.88,3.20
